In [2]:
'''
generate dataset from qplib
from Xi Gao
'''

import osqp
import time
import tqdm
import random
import pickle
import numpy as np
import gurobipy as gp

from gurobipy import Model, GRB, QuadExpr, LinExpr
from scipy.sparse import csc_matrix

## Import Data

In [34]:
# qp_path = ''
# m = gp.read('/home/jxxiong/A-xjx/DC3/datasets/dcopf/dcopf_model.lp')
m = gp.read('/home/jxxiong/A-xjx/DC3/datasets/qplib/QPLIB_8906.lp')

Read LP format model from file /home/jxxiong/A-xjx/DC3/datasets/dcopf/dcopf_model.lp
Reading time = 0.01 seconds
obj: 936 rows, 484 columns, 2244 nonzeros


In [35]:
#quadratic matrix
ini_q_matrix = m.getQ().toarray()

#linear vector
ini_p_vec = []
variables = m.getVars()
for var in variables:
    ini_p_vec.append(var.obj)
ini_p_vec = np.array(ini_p_vec)

print('The shape of quadratic matrix: {}.'.format(ini_q_matrix.shape))
print('The shape of coefficient vector: {}.'.format(ini_p_vec.shape))

The shape of quadratic matrix: (484, 484).
The shape of coefficient vector: (484,).


In [36]:
ini_lb = []
ini_ub = []
for var in variables:
    ini_lb.append(var.lb)
    ini_ub.append(var.ub)
ini_lb = np.array(ini_lb)
ini_ub = np.array(ini_ub)
print('The shape of lower bound vector: {}.'.format(ini_lb.shape))
print('The shape of upper bound vector: {}.'.format(ini_ub.shape))

The shape of lower bound vector: (484,).
The shape of upper bound vector: (484,).


In [37]:
#inequality and equality coefficient matrix
ini_ineq_matrix = []
ini_ineq_rhs = []
ini_eq_matrix = []
ini_eq_rhs = []
a_matrix = m.getA().toarray()

i = 0
for constr in m.getConstrs():
    if constr.sense == '<':
        ini_ineq_matrix.append(a_matrix[i,:])
        ini_ineq_rhs.append(constr.RHS)
        i += 1
    elif constr.sense == '>':
        ini_ineq_matrix.append(-a_matrix[i,:])
        ini_ineq_rhs.append(-constr.RHS)
        i += 1
    elif constr.sense == '=':
        ini_eq_matrix.append(a_matrix[i,:])
        ini_eq_rhs.append(constr.RHS)
        i += 1

ini_ineq_matrix = np.array(ini_ineq_matrix)
ini_ineq_rhs = np.array(ini_ineq_rhs)
ini_eq_matrix = np.array(ini_eq_matrix)
ini_eq_rhs = np.array(ini_eq_rhs)

print('The shape of inequality coefficient matrix: {}.'.format(ini_ineq_matrix.shape))
print('The shape of inequality RHS vector: {}.'.format(ini_ineq_rhs.shape))
print('The shape of equality coefficient matrix: {}.'.format(ini_eq_matrix.shape))
print('The shape of equality RHS vector: {}.'.format(ini_eq_rhs.shape))

The shape of inequality coefficient matrix: (490, 484).
The shape of inequality RHS vector: (490,).
The shape of equality coefficient matrix: (446, 484).
The shape of equality RHS vector: (446,).


In [38]:
num_var = ini_q_matrix.shape[0]
num_ineq = ini_ineq_matrix.shape[0]
num_eq = ini_eq_matrix.shape[0]

print('The number of variables: {}.'.format(num_var))
print('The number of inequality constraints: {}.'.format(num_ineq))
print('The number of equality constraints: {}.'.format(num_eq))

The number of variables: 484.
The number of inequality constraints: 490.
The number of equality constraints: 446.


## Random Sample

### Randomly sample the variables, ineq cons and eq cons to perturb

In [39]:
# num_chosed_var = 1500
# # num_chosed_ineq = 100
# num_chosed_eq = 200

# chosed_var = sorted(random.sample(range(0, num_var), num_chosed_var))
# # chosed_ineq = sorted(random.sample(range(0, num_ineq), num_chosed_ineq))
# chosed_eq = sorted(random.sample(range(0, num_eq), num_chosed_eq))

In [40]:
num_chosed_var = num_var
num_chosed_ineq = num_ineq
num_chosed_eq = num_eq

# chosed_eq = sorted(random.sample(range(0, num_eq), num_chosed_eq))

In [41]:
q_matrix = ini_q_matrix.copy()
p_vec = ini_p_vec.copy()
ineq_matrix = ini_ineq_matrix.copy()

ineq_rhs = ini_ineq_rhs.copy()

eq_matrix = ini_eq_matrix.copy()
eq_rhs = ini_eq_rhs.copy()
nonzero_indices = np.nonzero(ini_eq_rhs)
nonzero_values = ini_eq_rhs[nonzero_indices]
perturbed_values = nonzero_values * (1 + np.random.uniform(-0.3, 0.3, size=nonzero_values.shape))
eq_rhs[nonzero_indices] = perturbed_values
# eq_rhs = eq_rhs[chosed_eq]

In [42]:
def perturb_eq_rhs(ini_eq_rhs, num_samples):
    eq_rhs_list = []
    for _ in range(num_samples):
        eq_rhs = ini_eq_rhs.copy()
        nonzero_indices = np.nonzero(ini_eq_rhs)
        nonzero_values = ini_eq_rhs[nonzero_indices]
        perturbed_values = nonzero_values * (1 + np.random.uniform(-0.1, 0.1, size=nonzero_values.shape))
        # eq_rhs += np.random.uniform(-0.3, 0.3, size=eq_rhs.shape)
        eq_rhs[nonzero_indices] = perturbed_values
        eq_rhs_list.append(eq_rhs)
    return np.array(eq_rhs_list)

In [43]:
def solve_gurobi(Q, p, A, x, G, h, lb, ub):
    # Create a new Gurobi model
    model = Model("qp")

    # Add variables to the model
    n_vars = Q.shape[0]
    vars = []
    for i in range(n_vars):
        vars.append(model.addVar(lb=lb[i], ub=ub[i], vtype=GRB.CONTINUOUS, name=f"x_{i}"))
        # vars.append(model.addVar(vtype=GRB.CONTINUOUS, name=f"x_{i}"))


    # Set the objective function
    obj = QuadExpr()
    for i in range(n_vars):
        for j in range(n_vars):
            if Q[i, j] != 0:
                obj.add(vars[i] * vars[j] * Q[i, j]* 0.5)

    for i in range(n_vars):
        if p[i] != 0:
            obj.add(vars[i] * p[i])

    model.setObjective(obj, GRB.MINIMIZE)

    # Add equality constraints
    for i in range(A.shape[0]):
        expr = LinExpr()
        for j in range(n_vars):
            if A[i, j] != 0:
                expr.add(vars[j] * A[i, j])
        model.addConstr(expr == x[i])
    
    # Add inequality constraints
    for i in range(G.shape[0]):
        expr = LinExpr()
        for j in range(n_vars):
            if G[i, j] != 0:
                expr.add(vars[j] * G[i, j])
        model.addConstr(expr <= h[i])
    
    model.setParam('FeasibilityTol', 1e-4)
    # Solve the model
    model.optimize()
    
    # Retrieve the results
    if model.status == GRB.OPTIMAL:
        sol_gurobi = np.array([var.x for var in vars])
        return sol_gurobi
    else:
        print("No optimal solution found.")

In [44]:
def solve_osqp(Q, p, A, x, G, h, lb, ub):
    solver = osqp.OSQP()
    my_A = np.vstack([A, G, np.diag(np.ones(Q.shape[0]))])
    my_l = np.hstack([x, -np.ones(h.shape[0]) * np.inf, lb])
    my_u = np.hstack([x, h, ub])
    solver.setup(P=csc_matrix(Q), q=p, A=csc_matrix(my_A), l=my_l, u=my_u, verbose=True, eps_prim_inf=1e-10, eps_dual_inf=1e-10, eps_abs=1e-10, eps_rel=1e-10)
    results_osqp = solver.solve()
    sol_osqp = np.array(results_osqp.x)

    return sol_osqp

In [20]:
from dcopf_utils import DcopfProblem
filepath = "/home/jxxiong/A-xjx/DC3/datasets/dcopf/dcopf_data"
with open(filepath, 'rb') as f:
    dataset = pickle.load(f)
data = DcopfProblem(dataset)

In [27]:
Q, p, A, G, h = data.Q_np, data.p_np, data.A_np, data.G_np, data.h_np
x = data.X[0].detach().cpu().numpy()
lb = data.Lb.detach().cpu().numpy()
ub = data.Ub.detach().cpu().numpy()

In [30]:
solve_osqp(Q, p, A, x, G, h, lb, ub)

-----------------------------------------------------------------
           OSQP v0.6.3  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2021
-----------------------------------------------------------------
problem:  variables n = 484, constraints m = 1420
          nnz(P) + nnz(A) = 2759
settings: linear system solver = qdldl,
          eps_abs = 1.0e-10, eps_rel = 1.0e-10,
          eps_prim_inf = 1.0e-10, eps_dual_inf = 1.0e-10,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 4000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: off, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -1.9096e+04   1.70e+00   1.70e+05   1.00e-01   9.79e-04s
 200   1.3259e+04   3.60e-03   8.34e+00   1.00e-01   7.42e-03s
 400   1.3119e+04   2.15e-04   1.03e-01   1

array([-1.06108156e-305,  2.72000001e-002,  2.31700000e-001,
        8.38000001e-002,  1.33920000e+000,  3.90200000e-001,
        8.38000001e-002,  1.36000001e-002,  1.50000001e-002,
        1.62000001e-002,  5.25000001e-002,  2.82000001e-002,
        3.36098888e+000,  8.38000001e-002,  2.82000001e-002,
        8.38000001e-002,  2.31700000e-001,  1.20000001e-002,
        1.36000001e-002,  1.33920000e+000,  2.31700000e-001,
        1.41000001e-002,  7.98000001e-002,  2.31700000e-001,
        1.36000001e-002,  8.38000001e-002,  5.40000001e-002,
        7.20000009e-003,  1.36000001e-002,  3.90200000e-001,
        8.38000001e-002,  3.90200000e-001,  9.60000007e-003,
       -1.01207745e-001, -1.02154297e-001, -1.38562699e-001,
       -1.38524539e-001, -1.71725711e-001, -1.83375244e-001,
       -1.98391162e-001, -1.98024060e-001, -9.62797168e-002,
       -1.07929288e-001, -1.57683575e-001, -1.34537416e-001,
       -3.07458797e-002, -2.63048343e-002, -1.01634433e-001,
       -1.01634433e-001,

1375   1.3127e+04   8.99e-11   5.30e-08   1.00e-01   2.55e-02s

status:               solved
number of iterations: 1375
optimal objective:    13126.5495
run time:             2.56e-02s
optimal rho estimate: 2.35e-01



In [47]:
Y = solve_gurobi(ini_q_matrix, ini_p_vec, ini_eq_matrix, ini_eq_rhs, ini_ineq_matrix, ini_ineq_rhs, ini_lb, ini_ub)

Set parameter FeasibilityTol to value 0.0001
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.2 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i9-12900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 936 rows, 484 columns and 2244 nonzeros
Model fingerprint: 0xccf9b554
Model has 31 quadratic objective terms
Coefficient statistics:
  Matrix range     [4e-01, 3e+02]
  Objective range  [1e+00, 2e+03]
  QObjective range [4e+01, 4e+01]
  Bounds range     [7e-03, 7e+00]
  RHS range        [3e-03, 9e-01]
Presolve removed 771 rows and 338 columns
Presolve time: 0.00s
Presolved: 165 rows, 196 columns, 721 nonzeros
Presolved model has 31 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.538e+03
 Factor NZ  : 4.102e+03
 Factor Ops : 1.144e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter     

In [46]:
_ = solve_osqp(ini_q_matrix, ini_p_vec, ini_eq_matrix, ini_eq_rhs, ini_ineq_matrix, ini_ineq_rhs, ini_lb, ini_ub)

-----------------------------------------------------------------
           OSQP v0.6.3  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2021
-----------------------------------------------------------------
problem:  variables n = 484, constraints m = 1420
          nnz(P) + nnz(A) = 2759
settings: linear system solver = qdldl,
          eps_abs = 1.0e-10, eps_rel = 1.0e-10,
          eps_prim_inf = 1.0e-10, eps_dual_inf = 1.0e-10,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 4000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: off, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -2.1588e+04   1.76e+00   1.43e+05   1.00e-01   7.70e-04s
 200   1.3271e+04   3.10e-04   3.74e+00   1.00e-01   5.38e-03s
 400   1.3283e+04   7.59e-06   3.98e-02   1

# Gurobi v.s. OSQP

## Random generate data

In [55]:
num_var = 200
num_ineq = 100
num_eq = 100

num_chosed_var = 200
num_chosed_ineq = 100
num_chosed_eq = 100

model = gp.Model()
q_matrix = np.diag(np.random.random(num_var))
p_vec = np.random.random(num_var)
eq_matrix = np.random.normal(loc=0, scale=1., size=(num_eq, num_var))
eq_rhs = np.random.uniform(-1, 1, size=(num_eq))
ineq_matrix = np.random.normal(loc=0, scale=1., size=(num_ineq, num_var))
ineq_rhs = np.sum(np.abs(ineq_matrix@np.linalg.pinv(eq_matrix)), axis=1)

## osqp

In [56]:
Q = q_matrix
p = p_vec
Xi = eq_rhs #X[0]
A = eq_matrix
G = ineq_matrix
h = ineq_rhs

solver = osqp.OSQP()
my_A = np.vstack([A, G])
my_l = np.hstack([Xi, -np.ones(h.shape[0]) * np.inf])
my_u = np.hstack([Xi, h])
solver.setup(P=csc_matrix(Q), q=p, A=csc_matrix(my_A), l=my_l, u=my_u, verbose=True, eps_prim_inf=1e-10, eps_dual_inf=1e-10, eps_abs=1e-10, eps_rel=1e-10)
results_osqp = solver.solve()
sol_osqp = np.array(results_osqp.x)

-----------------------------------------------------------------
           OSQP v0.6.3  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2021
-----------------------------------------------------------------
problem:  variables n = 200, constraints m = 200
          nnz(P) + nnz(A) = 40200
settings: linear system solver = qdldl,
          eps_abs = 1.0e-10, eps_rel = 1.0e-10,
          eps_prim_inf = 1.0e-10, eps_dual_inf = 1.0e-10,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 4000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: off, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -2.1100e+01   1.59e+00   3.44e+02   1.00e-01   1.49e-02s
 150  -3.2332e+01   1.21e-09   2.88e-12   7.83e-03   3.82e-02s

status:               solved
number of ite

## Gurobi

In [59]:
# Create a new Gurobi model
model = Model("qp")

# Add variables to the model
n_vars = Q.shape[0]
vars = []
for i in range(n_vars):
    vars.append(model.addVar(lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name=f"x_{i}"))

# Set the objective function
obj = QuadExpr()
for i in range(n_vars):
    for j in range(n_vars):
        if Q[i, j] != 0:
            obj.add(vars[i] * vars[j] * Q[i, j]* 0.5)

for i in range(n_vars):
    if p[i] != 0:
        obj.add(vars[i] * p[i])

model.setObjective(obj, GRB.MINIMIZE)

# Add equality constraints
for i in range(A.shape[0]):
    expr = LinExpr()
    for j in range(n_vars):
        if A[i, j] != 0:
            expr.add(vars[j] * A[i, j])
    model.addConstr(expr == Xi[i])

# Add inequality constraints
for i in range(G.shape[0]):
    expr = LinExpr()
    for j in range(n_vars):
        if G[i, j] != 0:
            expr.add(vars[j] * G[i, j])
    model.addConstr(expr <= h[i])

model.setParam('FeasibilityTol', 1e-4)
# Solve the model
model.optimize()

# Retrieve the results
if model.status == GRB.OPTIMAL:
    sol_gurobi = np.array([var.x for var in vars])
    # print("Optimal solution:", sol_gurobi)
else:
    print("No optimal solution found.")


Set parameter FeasibilityTol to value 0.0001
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.2 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i9-12900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 200 rows, 200 columns and 40000 nonzeros
Model fingerprint: 0x19bdb544
Model has 200 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-05, 4e+00]
  Objective range  [5e-03, 1e+00]
  QObjective range [5e-03, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+01]


Presolve time: 0.01s
Presolved: 200 rows, 200 columns, 40000 nonzeros
Presolved model has 200 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 200
 AA' NZ     : 1.990e+04
 Factor NZ  : 2.010e+04
 Factor Ops : 2.687e+06 (less than 1 second per iteration)
 Threads    : 24

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.88997315e-01 -5.62820695e-02  4.72e-16 3.39e+00  1.00e+06     0s
   1   4.61711078e+06 -4.64117283e+06  1.12e+02 2.92e+01  1.93e+05     0s
   2   1.38901967e+06 -1.40217605e+06  3.42e-12 3.83e-07  2.79e+04     0s
   3   2.01947267e+05 -2.06983126e+05  3.02e-12 1.25e-12  4.09e+03     0s
   4   2.89870671e+04 -3.09155310e+04  7.68e-13 3.72e-13  5.99e+02     0s
   5   4.01315417e+03 -4.75336747e+03  3.00e-13 1.62e-13  8.77e+01     0s
   6   4.93405893e+02 -7.81371583e+02  1.37e-13 1.08e-13  1.27e+01     0s
   7   2.83461339e+01 -1.49562544e+02  6.19e-14 2.0

In [ ]:
def val(Q, p, A, x, G, h, y):
    obj = 0.5 * y.T @ Q @ y + p @ y
    eq = np.linalg.norm(A @ y - x)
    ineq = np.linalg.norm(np.maximum(G @ y - h, 0))
    return obj, eq, ineq

In [ ]:
val(Q, p, A, Xi, G, h, sol_gurobi)

(-40.53424343951364, 4.153896145489478e-14, 0.0)

In [ ]:
val(Q, p, A, Xi, G, h, sol_osqp)

(-40.53424347067941, 3.308438273888836e-12, 3.5389125674729555e-10)